In [1]:
from tqdm.auto import tqdm 
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nf
from netCDF4 import Dataset
%matplotlib inline
import glob
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [2]:
datapath="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/Data/ECMWF/ERA5_25kmx3hr/"
path="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/wp/"
output="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/create_ts/outputs/"
target="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/create_ts/outputs/targets/"

In [103]:
track = sorted(glob.glob(path+'wp_2016.csv'))

In [104]:
tracksDF = pd.read_csv(track[0])

In [105]:
tracksDF['name'].unique()

array(['NEPARTAK', 'OMAIS', 'CONSON', 'CHANTHU', 'MINDULLE', 'LIONROCK',
       'MERANTI', 'MALAKAS', 'MEGI', 'SONGDA', 'AERE', 'MEARI',
       'NOCK-TEN'], dtype=object)

In [106]:
stormnames = list(tracksDF['name'].unique())

In [107]:
[str(tracksDF[tracksDF['name']==stormnames[0]].time[i]).split(':')[0] for i in range(len(tracksDF[tracksDF['name']==stormnames[0]].time))]

['2016-07-02',
 '2016-07-02',
 '2016-07-02',
 '2016-07-02',
 '2016-07-03',
 '2016-07-03',
 '2016-07-03',
 '2016-07-03',
 '2016-07-03',
 '2016-07-03',
 '2016-07-03',
 '2016-07-03',
 '2016-07-04',
 '2016-07-04',
 '2016-07-04',
 '2016-07-04',
 '2016-07-04',
 '2016-07-04',
 '2016-07-04',
 '2016-07-04',
 '2016-07-05',
 '2016-07-05',
 '2016-07-05',
 '2016-07-05',
 '2016-07-05',
 '2016-07-05',
 '2016-07-05',
 '2016-07-05',
 '2016-07-06',
 '2016-07-06',
 '2016-07-06',
 '2016-07-06',
 '2016-07-06',
 '2016-07-06',
 '2016-07-06',
 '2016-07-06',
 '2016-07-07',
 '2016-07-07',
 '2016-07-07',
 '2016-07-07',
 '2016-07-07',
 '2016-07-07',
 '2016-07-07',
 '2016-07-07',
 '2016-07-08',
 '2016-07-08',
 '2016-07-08',
 '2016-07-08',
 '2016-07-08',
 '2016-07-08',
 '2016-07-08',
 '2016-07-08',
 '2016-07-09',
 '2016-07-09',
 '2016-07-09',
 '2016-07-09',
 '2016-07-09',
 '2016-07-09',
 '2016-07-09',
 '2016-07-09',
 '2016-07-10']

In [45]:
def largearea_withpres(dataset=None,invar=None,indices=None,tc_irad=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    rgspt=40
    var_out=np.zeros((len(indices),len(dataset.plev.data),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        for ip in range(len(dm2.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out  

In [46]:
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    rgspt=40
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out


In [12]:
#for dm1 files functions
def make_timeseries_step1(TCname=None,era5_date=None,era5_hour=None):
    track=tracksDF[tracksDF['name']==TCname].reset_index()
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-5
    tc_irad[:,1] = pos[:,0]+5
    tc_irad[:,2] = pos[:,1]-5
    tc_irad[:,3] = pos[:,1]+5
    return pos,indices_store,tc_irad
    

In [11]:
def readyear_automatic(year=None):
    dm2 = xr.open_dataset(datapath+'/mslp/mslp_'+str(year)+'.nc')
    #tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/nio/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,era5_date,era5_hour

dm2,era5_date,era5_hour = readyear_automatic(2014)

In [12]:
TCslp_ts = []
for TCname in tqdm(stormnames):
    ###########################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['mslp']
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmin(obj) for obj in tslist]
    #############################################################################################
    TCslp_ts.append(tsdict)
    

  0%|          | 0/14 [00:00<?, ?it/s]

In [13]:
storeTCdicts = {}
for ind in range(len(TCslp_ts)):
    storeTCdicts[stormnames[ind]] = pd.DataFrame.from_dict(TCslp_ts[ind])

In [14]:
for ind,obj in enumerate(stormnames):
    storeTCdicts[obj].to_csv(output+'2014_pmin_wpac_'+str(obj)+'.csv')

10m Wind

In [108]:
def readyear_automatic(year=None):
    dm2 = xr.open_dataset(datapath+'/wind10/wind10_'+str(year)+'.nc')
    #tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/nio/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,era5_date,era5_hour

dm2,era5_date,era5_hour = readyear_automatic(2016)

In [109]:
TCws10_ts = []
for TCname in tqdm(stormnames):
    ###########################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    
    ################################################################################################
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['wind10']
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmax(obj) for obj in tslist]
    #############################################################################################
    TCws10_ts.append(tsdict)

  0%|          | 0/13 [00:00<?, ?it/s]

In [110]:
storeTCdicts = {}
for ind in range(len(TCws10_ts)):
    storeTCdicts[stormnames[ind]] = pd.DataFrame.from_dict(TCws10_ts[ind])

In [111]:
for ind,obj in enumerate(stormnames):
    storeTCdicts[obj].to_csv(output+'2016_wmax_wpac_'+str(obj)+'.csv')

script ends here